In [1]:
from dataclasses import dataclass
import time
from tqdm import tqdm
from multiprocessing import Pool
from sklearn import preprocessing

import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, SecondLocator, DateFormatter
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import obspy as op
from obspy import read,read_inventory, UTCDateTime, Stream, Trace
from obspy.clients.fdsn.client import Client
from obspy.signal.rotate import rotate_ne_rt
from obspy.geodetics import gps2dist_azimuth,kilometers2degrees
from obspy.taup import TauPyModel

import json
import glob
import os
import numpy as np
from itertools import combinations
import pandas as pd
from scipy.signal import spectrogram, detrend, resample,savgol_filter,decimate,hilbert
from scipy.stats import circmean, circstd

import pyarrow.feather as feather
import seaborn as sns

import datetime

from sklearn.linear_model import LinearRegression,HuberRegressor,TheilSenRegressor

In [14]:
# ==================
# Configuration file
# ==================

# Folders input

EVENT_DIR = '/medata03/SEISCOMP_DATA/'

XML_DIR = '/home/sysop/dados_posdoc/PROJETO_RSIS/XML/'

# Folders output

ORIENTATION_OUTPUT = '/home/sysop/dados_posdoc/PROJETO_RSIS/ORIENTATION_OUTPUT/'

# -------------------------------

# Stations and OBSs information

#STATION_LST = ['VAS01','DUB01','CAM01','MAN01','ALF01','RIB01','GUA01','CMC01','GDU01','ANA01','TER01','MAJ01']
STATION_LST = ['NAN01']
PERIOD_BANDS = [0.02,0.5]


# -------------------------------

# create figures?

VERBOSE = True

# Input parameters

FIRSTDAY = '2014-01-01'
LASTDAY = '2016-12-31'

# default parameters to define the signal and noise windows used to estimate the SNR:
# - the signal window is defined according to time after Rayleigh-wave arrival:

TIME_WINDOW = 30

# Rayleigh-wave time windows start
TIME_START_P_REGIONAL = 3

# Rayleigh-wave time windows final
TIME_FINAL_P_REGIONAL = 12

# min magnitude
minmagnitude = 6

# distance (MIN & MAX)
GCARC_MIN = 5
GCARC_MAX = 100

# -------------------------------
# Mappoing parameters

LLCRNRLON_LARGE = -50
URCRNRLON_LARGE = -38
LLCRNRLAT_LARGE = -30
URCRNRLAT_LARGE = -12

# -------------------------------

# Constants and parameters

ONEDAY = datetime.timedelta(days=1)

# -------------------------------

# MULTIPROCESSING

num_processes = 20

# =================
# Filtering by date
# =================

fday = UTCDateTime(FIRSTDAY)
lday = UTCDateTime(LASTDAY)
INTERVAL_PERIOD = [UTCDateTime(x.astype(str)) for x in np.arange(fday.datetime,lday.datetime+ONEDAY,ONEDAY)]
INTERVAL_PERIOD_DATE = [str(x.year)+'.'+"%03d" % x.julday for x in INTERVAL_PERIOD]

In [8]:
# =========
# Functions
# =========

def rms(x):
    """
    Function to calculate root-mean-square of array

    Parameters
    ----------
    x : :class:`~numpy.ndarray`
        Input array

    Returns
    -------
    rms : float
        Root-Mean-Square value of `x`
    """

    return np.sqrt(np.mean(x**2))

def energy(x):
    """
    Function to calculate energy of array

    Parameters
    ----------
    x : :class:`~numpy.ndarray`
        Input array

    Returns
    -------
    energy : float
        Square value of `x`
    """

    return np.sum(x**2)
    
# --------------------------------------------------------------------------
def Braunmiller_algorithm(tr1,tr2,trZ,ang,dphi):
    
    '''
    The P-wave particle motion in an isotropic, homogeneous layered medium is in the direction of the seismic ray connecting
    source and receiver along a great circle path. The horizontal
    projection of the path from sensor to source defines the radial
    direction and its angle with respect to true north is the back
    azimuth. The P-wave energy in this medium is confined to the
    vertical and radial seismogram components and zero on the
    transverse component. We use this principle to estimate the
    back azimuth. 
    
    The sensor 'misorientation angle' is the difference
    between true and empirically estimated back azimuth with
    positive values for clockwise sensor misorientation.

    == Quality criteria for automatic processing ==
    For selecting reliable back azimuths, we implemented three quality criteria
     - (1) overall signal strength;
     - (2) similarity of vertical and radial components; and
     - (3) low transverse-to-radial energy ratio.
     - (4) low radial-to-vertical energy ratio.
     
    In addition, we required a minimum epicenter distance to minimize potential 
    systematic earthquake mislocation effects on back azimuths.
    '''
            
    # Criterias
       
    signal_strength = np.zeros(len(ang))
    similarity_ZR = np.zeros(len(ang))
    energy_ratio_TR = np.zeros(len(ang))
    energy_ratio_RZ = np.zeros(len(ang))
    
    # Search through azimuths from 0 to 180 deg and find best-fit azimuth                    
    for k, an in enumerate(ang):
        R, T = rotate_ne_rt(tr1, tr2, an)
        
        # Cross-correlation coefficient between vertical and radial component
        similarity_ZR[k] = np.corrcoef(trZ,R)[0, 1]
        
        # Measure of the transverse to radial ratio
        energy_ratio_TR[k] = 1. - energy(T)/energy(R)
        
        # Measure of the radial to vertical ratio
        energy_ratio_RZ[k] = 1. - energy(R)/energy(trZ)
        
        signal_strength[k] = energy(T)

    # Normalizing data
    signal_strength = (signal_strength - np.min(signal_strength)) / (np.max(signal_strength) - np.min(signal_strength)) 
    energy_ratio_TR = (energy_ratio_TR - np.min(energy_ratio_TR)) / (np.max(energy_ratio_TR) - np.min(energy_ratio_TR)) 
    energy_ratio_RZ = (energy_ratio_RZ - np.min(energy_ratio_RZ)) / (np.max(energy_ratio_RZ) - np.min(energy_ratio_RZ)) 
    similarity_ZR = (similarity_ZR - np.min(similarity_ZR)) / (np.max(similarity_ZR) - np.min(similarity_ZR)) 
                    
    return similarity_ZR,energy_ratio_TR,energy_ratio_RZ,signal_strength

#-------------------------------------------------------------------------------

In [4]:
# ============
# Main program
# ============
print('==================')
print('Downloading events')
print('==================')
print('\n')

client = Client('IRIS')

cat = client.get_events(starttime=fday, endtime=lday, minmagnitude=minmagnitude)
print(cat)



471 Event(s) in Catalog:
2016-12-29T22:30:19.300000Z |  -9.028, +118.664 | 6.3  Mww
2016-12-25T14:22:27.030000Z | -43.405,  -73.940 | 7.6  Mww
...
2014-01-13T04:01:04.530000Z | +18.992,  -66.868 | 6.3  MW
2014-01-01T16:03:29.770000Z | -13.877, +167.272 | 6.5  MW
To see all events call 'print(CatalogObject.__str__(print_all=True))'


In [15]:
SNR_MIN = 10 #db
CCVR_MIN = 0.7
TRR_MIN = 0.7
RVR_MIN = 0.2

def calculate_orientation(sta):
    
    # ---------------------------
    # Retrieving events waveforms
    # ---------------------------
    
    for event in tqdm(cat, total=len(cat),desc=sta+' orientation'):
        # ------------------------------
        # Check if the event is eligible

        depth_event = event.origins[-1].depth/1000
        event_time = event.origins[-1].time
        event_name = event.origins[-1].time.strftime('%Y.%j.%H.%M.%S')

        year = event.origins[-1].time.strftime('%Y')
        julian_day = event.origins[-1].time.strftime('%j')

        network = 'ON'
        station = sta    
                   
        # ---------------
        # Import XML file
                        
        xml_file = glob.glob(XML_DIR+network+'.'+sta+'*')[0]
        station_xml = op.read_inventory(xml_file)
                            
        # Epicentral distance:
            
        sta_lon = station_xml[-1][-1][-1].longitude
        sta_lat = station_xml[-1][-1][-1].latitude
            
        ev_lat = event.origins[-1].latitude
        ev_lon = event.origins[-1].longitude
            
        dist_pair,az_pair,baz_pair = gps2dist_azimuth(ev_lat, ev_lon,sta_lat, sta_lon)
        gcarc_pair = kilometers2degrees(dist_pair/1000)
        baz_pair_round = round(baz_pair)
        gcarc_pair_round = round(gcarc_pair)
        dist_pair_round = round(dist_pair)  

        # Taup: theoretical travel times 
        model = TauPyModel(model="iasp91")
        arrivals = model.get_travel_times(source_depth_in_km=depth_event,distance_in_degree=gcarc_pair)
        
        # Event time + P arrival time
        event_time = event_time+arrivals[0].time
        
        if gcarc_pair > GCARC_MIN and gcarc_pair < GCARC_MAX:
            # ----------------------------
            # Check if feather file exists
            
            output_FEATHER_FILES_ORIENTATION = ORIENTATION_OUTPUT+'FEATHER_FILES/'+sta+'/'
            
            file_feather_name = output_FEATHER_FILES_ORIENTATION+sta+'_'+event_name+'_orientation_data.feather'
    
            station_pwd = glob.glob(EVENT_DIR+year+'/'+network+'/'+station+'/*')
    
            if os.path.isfile(file_feather_name):
                pass
        
            else:
                # -------------------------------
                # Check if components file exists
                        
                if (len([i for i in station_pwd if 'HHE.D' in i or 'HH2.D' in i]) > 0 and
                    len([i for i in station_pwd if 'HHN.D' in i or 'HH1.D' in i]) > 0 and
                    len([i for i in station_pwd if 'HHZ.D' in i]) > 0):
    
                    if (len(glob.glob([i for i in station_pwd if 'HHE.D' in i or 'HH2.D' in i][0]+'/*'+year+'.'+julian_day)) > 0 and
                        len(glob.glob([i for i in station_pwd if 'HHN.D' in i or 'HH1.D' in i][0]+'/*'+year+'.'+julian_day)) > 0 and
                        len(glob.glob([i for i in station_pwd if 'HHZ.D' in i][0]+'/*'+year+'.'+julian_day)) > 0):
                    
                        try:
                        
                            file_HHE = glob.glob([i for i in station_pwd if 'HHE.D' in i or 'HH2.D' in i][0]+'/*'+year+'.'+julian_day)[0]
                            file_HHN = glob.glob([i for i in station_pwd if 'HHN.D' in i or 'HH1.D' in i][0]+'/*'+year+'.'+julian_day)[0]
                            file_HHZ = glob.glob([i for i in station_pwd if 'HHZ.D' in i][0]+'/*'+year+'.'+julian_day)[0]
        
                            # --------
                            # Data HHE
                            
                            tr2_data_file = op.read(file_HHE)
                            tr2_data_file.trim(event_time-TIME_WINDOW,event_time+TIME_WINDOW)
                            tr2_data_file.taper(type='hann',max_percentage=0.1)
                            tr2_data_file.filter('bandpass',freqmin=PERIOD_BANDS[0],freqmax=PERIOD_BANDS[1],zerophase=True, corners=4)
                            tr2_data_filtered = tr2_data_file[0].data
                
                            # --------
                            # Data HHN
                
                            tr1_data_file = op.read(file_HHN)
                            tr1_data_file.trim(event_time-TIME_WINDOW,event_time+TIME_WINDOW)
                            tr1_data_file.taper(type='hann',max_percentage=0.1)
                            tr1_data_file.filter('bandpass',freqmin=PERIOD_BANDS[0],freqmax=PERIOD_BANDS[1],zerophase=True, corners=4)
                            tr1_data_filtered = tr1_data_file[0].data
                            
                            # --------
                            # Data HHZ
                            
                            trZ_data_file = op.read(file_HHZ)
                            trZ_data_file.trim(event_time-TIME_WINDOW,event_time+TIME_WINDOW)
                            trZ_data_file.taper(type='hann',max_percentage=0.1)
                            trZ_data_file.filter('bandpass',freqmin=PERIOD_BANDS[0],freqmax=PERIOD_BANDS[1],zerophase=True, corners=4)
                            trZ_data_filtered = trZ_data_file[0].data
                            trZ_time = trZ_data_file[0].times()-TIME_WINDOW       
                            
                            # -------------------------------------------------------------------------------------------------------------------------------
                            # Signal and noise windows
    
                            signal_window = (trZ_time >= -TIME_START_P_REGIONAL) & (trZ_time <= TIME_FINAL_P_REGIONAL)
                            noise_window = (trZ_time >= -(TIME_START_P_REGIONAL+TIME_FINAL_P_REGIONAL)) & (trZ_time <= -TIME_START_P_REGIONAL)
                            
                            noise = trZ_data_filtered[noise_window]
                            trZ_noise_time = trZ_time[noise_window]
    
                            tr2 = tr2_data_filtered[signal_window]
                            tr1 = tr1_data_filtered[signal_window]
                            trZ = trZ_data_filtered[signal_window]
                            trZ_signal_time = trZ_time[signal_window]
    
                            # -------------------------------------------------------------------------------------------------------------------------------
                            # Calculate and store SNR as attribute
                            
                            SNR = round(10.*np.log10(rms(trZ)**2/rms(noise)**2),1)
                                 
                            # -------------------------------------------------------------------------------------------------------------------------------
                            # Search Space of BAZ
                        
                            dphi = 0.1
                            ang = np.arange(0., 360., dphi)
                            
                            # -------------------------------------------------------------------------------------------------------------------------------
                            # Calculating criterias
                                            
                            SZR,ERTR,ERRZ,SS = Braunmiller_algorithm(tr1,tr2,trZ,ang,dphi)
                                           
                            # Find best index
                            cost_function = (
                                                SS +  # Minimizing
                                                (1 - SZR) +  # Maximizing similarity (equal to minimizing (1 - similarity))
                                                ERTR +  # Minimizing
                                                ERRZ  # Minimizing 
                                            )
                        
                            best_index = np.argmin(cost_function)
                            
                            # Get azimuth and correct for angles above 360
                            phi = round(ang[best_index])
                            orient = round(baz_pair - ang[best_index])
                            
                            # Get argument of maximum coherence:
                            SZR_max = SZR[best_index]
                            ERTR_max = ERTR[best_index]
                            ERRZ_max = ERRZ[best_index]
                            SS_max = SS[best_index]

                            # ----------------------------------------------------------------------------------------------------
                            
                            if VERBOSE == True:
                
                                # --------------------
                                # figure 
                                fig = plt.figure(figsize=(30, 10),constrained_layout=True)
                                fig.suptitle('Evento: '+event_name+'(Δ: '+str(gcarc_pair_round)+'° | M: '+str(event.magnitudes[-1].mag)+' '+event.magnitudes[-1].magnitude_type+' | D: '+str(round(depth_event))+' km) \n BAZ: '+str(baz_pair_round)+'° | PHI: '+str(phi)+'° | orient: '+str(orient)+'° | SNR: '+str(SNR)+' dB',fontsize=20)

                                # creating grid
                                gs = fig.add_gridspec(1, 2,width_ratios=[3,1])

                                gs0 = gs[0].subgridspec(3, 1)
                                gs1 = gs[1].subgridspec(4, 1)
                                
                                # Rotating components
                                new_R, new_T = rotate_ne_rt(tr1_data_filtered, tr2_data_filtered, phi)

                                # Transversal data
                                ax1 = fig.add_subplot(gs0[0, 0])
                                ax1.plot(trZ_time,new_T,'-k',lw=2)
                                ax1.plot(trZ_signal_time,tr2,c='gray',ls='--',lw=0.5)
                                ax1.annotate('Transversal', (0.9, 0.85),xycoords='axes fraction',fontsize=15, va='center',bbox=dict(boxstyle="round", fc="white"))
                                ax1.set_xlim(-TIME_WINDOW,TIME_WINDOW)
                                ax1.tick_params(axis="x", labelbottom=False)
                                ax1.axvline(x=-TIME_START_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)
                                ax1.axvline(x=TIME_FINAL_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)

                                # Radial data
                                ax2 = fig.add_subplot(gs0[1, 0], sharex=ax1, sharey=ax1)
                                ax2.plot(trZ_time,new_R,'-k')
                                ax2.plot(trZ_signal_time,tr1,c='gray',ls='--',lw=0.5)
                                ax2.annotate('Radial', (0.9, 0.85),xycoords='axes fraction',fontsize=15, va='center',bbox=dict(boxstyle="round", fc="white"))
                                ax2.tick_params(axis="x", labelbottom=False)
                                ax2.axvline(x=-TIME_START_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)
                                ax2.axvline(x=TIME_FINAL_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)

                                # Vertical data and noise and signal window
                                ax3 = fig.add_subplot(gs0[2, 0], sharex=ax1, sharey=ax1)
                                ax3.plot(trZ_time,trZ_data_filtered,'-k')
                                ax3.plot(trZ_noise_time,noise,'--b')
                                ax3.plot(trZ_signal_time,trZ,'--r')
                                ax3.annotate('Vertical', (0.9, 0.85),xycoords='axes fraction',fontsize=15, va='center',bbox=dict(boxstyle="round", fc="white"))
                                ax3.set_xlabel('Timelag (s)',fontsize=15)
                                ax3.axvline(x=-TIME_START_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)
                                ax3.axvline(x=TIME_FINAL_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)

                                # Transversal signal strength
                                ax4 = fig.add_subplot(gs1[0, 0])
                                ax4.plot(ang,SS,'.k')
                                ax4.plot(phi,SS_max,'*r',ms=10)
                                ax4.set_ylim(0,1)
                                ax4.tick_params(axis="x", labelbottom=False)
                                ax4.set_title('Transversal signal strength',fontsize=15)
                                
                                # Similarity between vertical and radial
                                ax5 = fig.add_subplot(gs1[1, 0], sharex=ax4)
                                ax5.plot(ang,SZR,'.k')
                                ax5.plot(phi,SZR_max,'*r',ms=10)
                                ax5.set_ylim(0,1)
                                ax5.set_xlim(0,360)
                                ax5.tick_params(axis="x", labelbottom=False)
                                ax5.set_title('Similarity between vertical and radial',fontsize=15)

                                # Transverse-to-Radial Energy Ratio
                                ax6 = fig.add_subplot(gs1[2, 0], sharex=ax5)
                                ax6.plot(ang,ERTR,'.k')
                                ax6.plot(phi,ERTR_max,'*r',ms=10)
                                ax6.tick_params(axis="x", labelbottom=False)
                                ax6.set_title('Transverse-to-Radial Energy Ratio',fontsize=15)
                                                                
                                # Radial-to-Vertical Energy Ratio
                                ax7 = fig.add_subplot(gs1[3, 0], sharex=ax6)
                                ax7.plot(ang,ERRZ,'.k')
                                ax7.plot(phi,ERRZ_max,'*r',ms=10)
                                ax7.set_title('Radial-to-Vertical Energy Ratio',fontsize=15)
                                ax7.set_xlabel('Orientation Angle (deg)',fontsize=15)

                                # --------------------------
                                # Adding global location map

                                ax_map = plt.axes([0.0, 0.82, 0.15, 0.15], projection=ccrs.Orthographic(central_latitude=sta_lat,central_longitude=sta_lon))
                                ax_map.set_global()

                                # ---------------------
                                # Adding background map 

                                ax_map.add_feature(cfeature.LAND)
                                ax_map.add_feature(cfeature.OCEAN)
                                ax_map.add_feature(cfeature.COASTLINE)
                            
                                ax_map.scatter(ev_lon,ev_lat,color="y",marker='*',s=200,ec='k',transform=ccrs.PlateCarree())
                                ax_map.scatter(sta_lon,sta_lat,color="r",marker='^',s=50,transform=ccrs.PlateCarree())
                                ax_map.plot([sta_lon, ev_lon], [sta_lat, ev_lat], c='gray',ls='-',lw=2, transform=ccrs.Geodetic())
        
                                if (SZR_max >= CCVR_MIN) & (SNR >= SNR_MIN) & (ERTR_max >= TRR_MIN) & (ERRZ_max <= RVR_MIN):
                                    label = 'good'
                                    
                                    # ----------------------------------------------------------------------------------------------------
                                    # Creating a Pandas DataFrame:
                                    column_info = [sta,event_name,dist_pair,gcarc_pair,baz_pair,SS_max,SZR_max,ERTR_max,ERRZ_max,SNR,phi,orient,label]
                                    columns_header = ['station','event','distance','gcarc','baz','signal_strength','similarity_vertical_radial','energy_transverse_radial','energy_radial_vertical','SNR','phi','orient','quality']
                                    orient_rayleigh_df = pd.DataFrame(column_info, index=columns_header).T
                        
                                    # ----------------------------------------------------------------------------------------------------
                                    # Convert from pandas to Arrow and saving in feather formart file
                                    os.makedirs(output_FEATHER_FILES_ORIENTATION,exist_ok=True)
                                    feather.write_feather(orient_rayleigh_df, file_feather_name)
                                else:
                                    label = 'bad'
                                    # ----------------------------------------------------------------------------------------------------
                                    # Creating a Pandas DataFrame:
                                    column_info = [sta,event_name,dist_pair,gcarc_pair,baz_pair,SS_max,SZR_max,ERTR_max,ERRZ_max,SNR,phi,orient,label]
                                    columns_header = ['station','event','distance','gcarc','baz','signal_strength','similarity_vertical_radial','energy_transverse_radial','energy_radial_vertical','SNR','phi','orient','quality']
                                    orient_rayleigh_df = pd.DataFrame(column_info, index=columns_header).T
                        
                                    # ----------------------------------------------------------------------------------------------------
                                    # Convert from pandas to Arrow and saving in feather formart file
                                    os.makedirs(output_FEATHER_FILES_ORIENTATION,exist_ok=True)
                                    feather.write_feather(orient_rayleigh_df, file_feather_name)
                                    
                                output_figure_ORIENTATION = ORIENTATION_OUTPUT+'ORIENTATION_FIGURES/EARTHQUAKES/'+sta+'/'
                                os.makedirs(output_figure_ORIENTATION,exist_ok=True)
                                fig.savefig(output_figure_ORIENTATION+'ORIENTATION_'+sta+'_'+event_name+'_'+label+'.png',dpi=300)
                                plt.close()
                        except:
                            pass


In [16]:
start_time = time.time()

with Pool(processes=20) as p:
    max_ = len(STATION_LST)
    with tqdm(total=max_) as pbar:
        for result in p.imap_unordered(calculate_orientation,STATION_LST):
            pbar.update()

print('\n')
print("--- %.2f execution time (min) ---" % ((time.time() - start_time)/60))
print('\n')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [06:54<00:00, 414.75s/it]



--- 6.92 execution time (min) ---


